In [ ]:
#| default_exp download

# Download helpers

- Download and process LLM-ready documents

In [ ]:
#| export
from fastcore.utils import *
from httpx import get
from fastcore.meta import delegates
from urllib.parse import urlparse, urljoin

In [ ]:
from IPython.display import Markdown,HTML
from fastcore.test import *

In [ ]:
#| export
def clean_md(text, rm_comments=True, rm_details=True):
    "Remove comments and `<details>` sections from `text`"
    if rm_comments: text = re.sub(r'\n?<!--.*?-->\n?', '', text, flags=re.DOTALL)
    if rm_details: text = re.sub(r'\n?<details>.*?</details>\n?', '', text, flags=re.DOTALL)
    return text

In [ ]:
#| export
@delegates(get)
def read_md(url, rm_comments=True, rm_details=True, **kwargs):
    "Read text from `url` and clean with `clean_docs`"
    return clean_md(get(url, **kwargs).text, rm_comments=rm_comments, rm_details=rm_details)

In [ ]:
mdurl = 'https://claudette.answer.ai/index.html.md'
md = read_md(mdurl)
# Markdown(md)

In [ ]:
#| export
def html2md(s:str, ignore_links=True):
    "Convert `s` from HTML to markdown"
    import html2text
    o = html2text.HTML2Text(bodywidth=5000)
    o.ignore_links = ignore_links
    o.mark_code = True
    o.ignore_images = True
    return o.handle(s)

In [ ]:
#| export
def read_html(url, # URL to read
              sel=None, # Read only outerHTML of CSS selector `sel`
              rm_comments=True, # Removes HTML comments
              rm_details=True, # Removes `<details>` tags
              multi=False, # Get all matches to `sel` or first one  
              wrap_tag=None, #If multi, each selection wrapped with <wrap_tag>content</wrap_tag>
              ignore_links=True,
             ): # Cleaned markdown
    "Get `url`, optionally selecting CSS selector `sel`, and convert to clean markdown"
    page = get(url).text
    if sel:
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(page, 'html.parser')
        if multi:
            page = [str(el) for el in soup.select(sel)]
            if not wrap_tag: page = "\n".join(page)
        else: page = str(soup.select_one(sel))
    mds = map(lambda x: clean_md(html2md(x, ignore_links=ignore_links), rm_comments, rm_details=rm_details), tuplify(page))
    if wrap_tag: return '\n'.join([f"\n<{wrap_tag}>\n{o}</{wrap_tag}>\n" for o in mds])
    else: return'\n'.join(mds)

In [ ]:
# test single class selector
listings = read_html('https://www.answer.ai/', sel='.listing-description')
assert len(listings) < 500

# Test multi class selector
listings = read_html('https://www.answer.ai/', sel='.listing-description', multi=True)
assert len(listings) > 1000 # returns more than single so selecting multi

# Test multi_wrap_tag
listings = read_html('https://www.answer.ai/', sel='.listing-description', multi=True, wrap_tag='document')
assert '<document>' in listings and '</document>' in listings 

In [ ]:
read_html('https://www.answer.ai/', sel='.listing-description', ignore_links=False)

'[ How I created a book chapter from video transcripts with SolveIt ](./posts/2025-10-13-video-to-doc.html)\n\n'

In [ ]:
# test tag css selectors
assert len(read_html('https://www.answer.ai/', sel='div.listing-description', multi=True)) > 1000
assert len(read_html('https://www.answer.ai/', sel='div', multi=True)) > 1000

In [ ]:
htmlurl = 'https://hypermedia.systems/hypermedia-a-reintroduction/'
hmd = read_html(htmlurl)
assert len(hmd) > 100
# Markdown(hmd)

In [ ]:
#| export
def get_llmstxt(url, optional=False, n_workers=None):
    "Get llms.txt file from and expand it with `llms_txt.create_ctx()`"
    if not url.endswith('llms.txt'): return None
    import llms_txt
    resp = get(url)
    if resp.status_code!=200: return None
    return llms_txt.create_ctx(resp.text, optional=optional, n_workers=n_workers)

In [ ]:
# print(get_llmstxt('https://llmstxt.org/llms.txt'))

In [ ]:
#| export
def split_url(url):
    "Split `url` into base, path, and file name, normalising name to '/' if empty"
    parsed = urlparse(url.strip('/'))
    base = f"{parsed.scheme}://{parsed.netloc}"
    path,spl,fname = parsed.path.rpartition('/')
    fname = spl+fname
    if not path and not fname: path='/'
    return base,path,fname

In [ ]:
urls = ('https://claudette.answer.ai/path/', 'https://claudette.answer.ai/', 'https://llmstxt.org', 'https://llmstxt.org/')

[split_url(o) for o in urls]

[('https://claudette.answer.ai', '', '/path'),
 ('https://claudette.answer.ai', '/', ''),
 ('https://llmstxt.org', '/', ''),
 ('https://llmstxt.org', '/', '')]

In [ ]:
#| export
def _tryget(url):
    "Return response from `url` if `status_code!=404`, otherwise `None`"
    res = get(url)
    return None if res.status_code==404 else url

In [ ]:
#| export
def find_docs(url):
    "If available, return LLM-friendly llms.txt context or markdown file location from `url`"
    base,path,fname = split_url(url)
    url = (base+path+fname).strip('/')
    if fname=='/llms.txt': return url
    if Path(fname).suffix in('.md', '.txt', '.rst'): return _tryget(url)
    if '.' in fname: return _tryget(url+'.md') or find_docs(url[:url.rfind('/')])
    res = _tryget(url+'/llms.txt')
    if res: return res
    res = _tryget(url+'/index.md')
    if res: return res
    res = _tryget(url+'/index.html.md')
    if res: return res
    res = _tryget(url+'/index-commonmark.md')
    if res: return res
    parsed_url = urlparse(url)
    if parsed_url.path == '/' or not parsed_url.path: return None
    return find_docs(urljoin(url, '..'))

In [ ]:
fl_url = 'https://answerdotai.github.io/fastlite'

In [ ]:
find_docs(fl_url)

'https://answerdotai.github.io/fastlite/llms.txt'

In [ ]:
#| eval: false
for o in urls: print(find_docs(o))

https://claudette.answer.ai/llms.txt


https://claudette.answer.ai/llms.txt


https://llmstxt.org/llms.txt


https://llmstxt.org/llms.txt


In [ ]:
#| eval: false
suffixes = ["/", "/tmp", "/tmp/tmp/"]
for suff in suffixes:
    for o in urls:  test_eq(find_docs(o), find_docs(o+suff))

test_eq(find_docs("https://github.com"), "https://github.com/llms.txt")
test_eq(find_docs("https://github.com/AnswerDotAI"), "https://github.com/llms.txt")
test_eq(find_docs("https://github.com/AnswerDotAI/"), "https://github.com/llms.txt")

In [ ]:
#| export
def read_docs(url, optional=False, n_workers=None, rm_comments=True, rm_details=True):
    "If available, return LLM-friendly llms.txt context or markdown file response for `url`"
    url = find_docs(url)
    if url.endswith('/llms.txt'): res = get_llmstxt(url, optional=optional, n_workers=n_workers)
    else: res = get(url).text
    return clean_md(res, rm_comments=rm_comments, rm_details=rm_details)

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()